In [23]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [35]:
btc = yf.download('BTC-USD', period='max',start='2019-01-01')
df = pd.DataFrame(btc)
df.reset_index(inplace=True) 
df['DayWeek'] = df['Date'].dt.dayofweek
df['start_point'] = 1000
df['start_point'] = df['start_point'].astype('float')
df = df[df['DayWeek'].isin([5, 2])]
df['DayWeek'] = df['DayWeek'].astype('int8')
df['action'] = df['DayWeek'].apply(lambda x: "Buy" if x == 5 else "Sell")
df.drop(columns=['Volume','Adj Close'], inplace=True)
df.to_csv('BTC.csv',index=False)
df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,DayWeek,start_point,action
1,2019-01-02,3849.216309,3947.981201,3817.409424,3943.409424,2,1000.0,Sell
4,2019-01-05,3851.973877,3904.903076,3836.900146,3845.194580,5,1000.0,Buy
8,2019-01-09,4031.552002,4068.403564,4022.662842,4035.296387,2,1000.0,Sell
11,2019-01-12,3686.973145,3698.978271,3653.810791,3661.301025,5,1000.0,Buy
15,2019-01-16,3631.509766,3685.777100,3624.673340,3655.006836,2,1000.0,Sell
...,...,...,...,...,...,...,...,...
2031,2024-07-24,65927.859375,67113.984375,65146.996094,65372.132812,2,1000.0,Sell
2034,2024-07-27,67911.812500,69398.507812,66705.218750,67813.335938,5,1000.0,Buy
2038,2024-07-31,66201.273438,66810.210938,64532.046875,64619.250000,2,1000.0,Sell
2041,2024-08-03,61414.808594,62148.371094,59836.527344,60680.093750,5,1000.0,Buy


In [36]:
df = pd.read_csv('BTC.csv')
df.reset_index(inplace=True)
if df.loc[0,'action'] == "Sell":
    df.loc[0,'action'] = "Hold"
for i in range(1, len(df)):
    if df.loc[i,'action'] == "Buy":
        df.loc[i+1,'start_point']= df.loc[i,'start_point']/df.loc[i,'Open']
        df.loc[i+1,"Dollar_balance"] = 0
        df.loc[i+1,"BTC_balance"] = df.loc[i+1,'start_point']

    elif df.loc[i,'action'] == "Sell":
        df.loc[i+1,'start_point'] = df.loc[i,'start_point']*df.loc[i,'Close']
        df.loc[i+1,"Dollar_balance"] = df.loc[i+1,'start_point']
        df.loc[i+1,"BTC_balance"] = 0

In [41]:
df.dropna(inplace=True)
df.to_excel("data_open_close.xlsx")
df

,index,Date,Open,High,Low,Close,DayWeek,start_point,action,Dollar_balance,BTC_balance
2,2.0,2019-01-09,4031.552002,4068.403564,4022.662842,4035.296387,2.0,0.259607,Sell,0.000000,0.259607
3,3.0,2019-01-12,3686.973145,3698.978271,3653.810791,3661.301025,5.0,1047.591836,Buy,1047.591836,0.000000
4,4.0,2019-01-16,3631.509766,3685.777100,3624.673340,3655.006836,2.0,0.284133,Sell,0.000000,0.284133
5,5.0,2019-01-19,3652.377930,3758.533447,3652.377930,3728.568359,5.0,1038.509144,Buy,1038.509144,0.000000
6,6.0,2019-01-23,3605.557129,3623.067871,3565.313965,3585.123047,2.0,0.284338,Sell,0.000000,0.284338
...,...,...,...,...,...,...,...,...,...,...,...
580,580.0,2024-07-24,65927.859375,67113.984375,65146.996094,65372.132812,2.0,0.337768,Sell,0.000000,0.337768
581,581.0,2024-07-27,67911.812500,69398.507812,66705.218750,67813.335938,5.0,22080.595501,Buy,22080.595501,0.000000
582,582.0,2024-07-31,66201.273438,66810.210938,64532.046875,64619.250000,2.0,0.325136,Sell,0.000000,0.325136
583,583.0,2024-08-03,61414.808594,62148.371094,59836.527344,60680.093750,5.0,21010.063909,Buy,21010.063909,0.000000


In [27]:
btc_df = df.drop(columns=['Open','High','Low','Close','DayWeek','start_point','Dollar_balance'])
s = btc_df['BTC_balance']
btc_df['BTC_balance'] = s.where(s>0)
btc_df.dropna(inplace=True)

dollar_df = df.drop(columns=['Open','High','Low','Close','DayWeek','start_point','BTC_balance'])
s = dollar_df['Dollar_balance']
dollar_df['Dollar_balance'] = s.where(s>0)
dollar_df.dropna(inplace=True)

In [42]:
fig = make_subplots(rows=1, cols=2)

btc_df['text'] = 'BTC Balance : ' + btc_df['BTC_balance'].round(2).astype(str) + 'BTC'
fig.add_trace(go.Scatter(x=btc_df['Date'], y=btc_df['BTC_balance'],text=btc_df['text'],name='BTC'),row=1, col=1)


dollar_df['text'] = 'Dollar Balance : ' + dollar_df['Dollar_balance'].astype(int).astype(str) + '$ '
fig.add_trace(go.Scatter(x=dollar_df['Date'], y=dollar_df['Dollar_balance'],text=dollar_df['text'],name='$'),row=1, col=2)

fig.update_layout(
    width=1300,
    height=500,
)
fig.show()

In [29]:
df = pd.read_csv('BTC.csv')
df.reset_index(inplace=True)
if df.loc[0,'action'] == "Sell":
    df.loc[0,'action'] = "Hold"
for i in range(1, len(df)):
    if df.loc[i,'action'] == "Buy":
        df.loc[i+1,'start_point']= df.loc[i,'start_point']/df.loc[i,'Open']
        df.loc[i+1,"Dollar_balance"] = 0
        df.loc[i+1,"BTC_balance"] = df.loc[i+1,'start_point']

    elif df.loc[i,'action'] == "Sell":
        df.loc[i+1,'start_point'] = df.loc[i,'start_point']*df.loc[i,'High']
        df.loc[i+1,"Dollar_balance"] = df.loc[i+1,'start_point']
        df.loc[i+1,"BTC_balance"] = 0

In [30]:
df.dropna(inplace=True)
df.to_excel("data_open_high.xlsx")
df

,index,Date,Open,High,Low,Close,DayWeek,start_point,action,Dollar_balance,BTC_balance
2,2.0,2019-01-09,4031.552002,4068.403564,4022.662842,4035.296387,2.0,2.596072e-01,Sell,0.000000e+00,0.259607
3,3.0,2019-01-12,3686.973145,3698.978271,3653.810791,3661.301025,5.0,1.056187e+03,Buy,1.056187e+03,0.000000
4,4.0,2019-01-16,3631.509766,3685.777100,3624.673340,3655.006836,2.0,2.864644e-01,Sell,0.000000e+00,0.286464
5,5.0,2019-01-19,3652.377930,3758.533447,3652.377930,3728.568359,5.0,1.055844e+03,Buy,1.055844e+03,0.000000
6,6.0,2019-01-23,3605.557129,3623.067871,3565.313965,3585.123047,2.0,2.890840e-01,Sell,0.000000e+00,0.289084
...,...,...,...,...,...,...,...,...,...,...,...
580,580.0,2024-07-24,65927.859375,67113.984375,65146.996094,65372.132812,2.0,1.558430e+02,Sell,0.000000e+00,155.843037
581,581.0,2024-07-27,67911.812500,69398.507812,66705.218750,67813.335938,5.0,1.045925e+07,Buy,1.045925e+07,0.000000
582,582.0,2024-07-31,66201.273438,66810.210938,64532.046875,64619.250000,2.0,1.540122e+02,Sell,0.000000e+00,154.012193
583,583.0,2024-08-03,61414.808594,62148.371094,59836.527344,60680.093750,5.0,1.028959e+07,Buy,1.028959e+07,0.000000


In [31]:
btc_df2 = df.drop(columns=['Open','High','Low','Close','DayWeek','start_point','Dollar_balance'])
s = btc_df2['BTC_balance']
btc_df2['BTC_balance'] = s.where(s>0)
btc_df2.dropna(inplace=True)
dollar_df2 = df.drop(columns=['Open','High','Low','Close','DayWeek','start_point','BTC_balance'])
s = dollar_df2['Dollar_balance']
dollar_df2['Dollar_balance'] = s.where(s>0)
dollar_df2.dropna(inplace=True)

In [32]:
fig = make_subplots(rows=1, cols=2)

btc_df2['text'] = 'BTC Balance : ' + btc_df2['BTC_balance'].round(2).astype(str) + 'BTC'
fig.add_trace(go.Scatter(x=btc_df2['Date'], y=btc_df2['BTC_balance'],name='BTC',text=btc_df2['text']),row=1, col=1)


dollar_df2['text'] = 'Dollar Balance : ' + dollar_df2['Dollar_balance'].astype(int).astype(str) + '$ '
fig.add_trace(go.Scatter(x=dollar_df2['Date'], y=dollar_df2['Dollar_balance'],name='$',text = dollar_df2['text']),row=1, col=2)

fig.update_layout(
    width=1300,
    height=500,
)
fig.show()